In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
#Define functions that identify each variable that we will be using. 
#Date function will be indexed after all season data is present.

#Define Player Name function
def PName(matches):
    PName = []
    for leg in matches:
        if leg[0][0] == 0:
            PName.append(leg[0][1])
        else:
            PName.append(leg[0][2])
    return PName

#Define Leg Code function
def LegCode(matches):
    LegCode = []
    for leg in matches:
        if leg[0][0] != 0:
            thisleg = leg[0][0]
            LegCode.append(leg[0][0])
        else:
            LegCode.append(thisleg)
    return LegCode

#Define Game Name function
def Game(matches):
    Game = []
    for leg in matches:
        if leg[0][0] != 0:
            thisgame = leg[0][1]
            Game.append(leg[0][1])
        else:
            Game.append(thisgame)
    return Game

#Define Player 3 Dart Average function
def P3DA(matches):
    P3DA = []
    for leg in matches:
        if leg[0][0] != 0:
            P3DA.append(leg[0][5])
        elif (leg[0][0] == 0) & (leg[0][4] == 0):
            P3DA.append(leg[0][2])
        else:
            P3DA.append(leg[0][4])
    for avg in P3DA:
        #301 legs with a zero average sometimes return cork value
        if (avg == 'C') or (avg == 'S'):
            avg = 0
    return P3DA

#Define Leg Win Function
def LegWin(matches):
    LegWin = []
    for leg in matches:
        if leg[0][0] != 0:
            LegWin.append(leg[0][9])
        elif (leg[0][0] == 0) & (leg[0][4] == 0):
            LegWin.append(leg[0][6])
        else:
            LegWin.append(leg[0][8])
    return LegWin


#Need a function to derive team name!


In [3]:
#Leg set function
def leggify(x):
    gmst = []
    for i in x:
        if (i.text == '') or (i.text == '\xa0'):
            gmst.append(0)
        else:
            gmst.append(i.get_text())
    return gmst

In [4]:
#Define the function that takes the ID of a match and then derives the game table using BeautifulSoup

def MatchTable(match):
    matchloc = 'https://members.dartconnect.com/history/report/players/match/' + str(match)
    summaryloc = 'https://members.dartconnect.com/history/report/match/' + str(match)
    matchpage = requests.get(matchloc)
    summarypage = requests.get(summaryloc)
    soup = BeautifulSoup(matchpage.text, 'lxml')
    psoup = BeautifulSoup(summarypage.text, 'lxml')
    
    #Find and set the date string
    datematch = soup.h2.next_element.next_element
    daydate = datematch.td.text
    day = daydate[0:3]
    pdate = daydate[-8:]
    if (pdate[0] == " "):
        date = '0' + pdate[1:]
    else:
        date = pdate
        
    #Identify game detail table
    game_pp = soup.find(class_="freeTable playerPref")    
    
    #Break table into rows
    rows = [row for row in game_pp.find_all("tr")]
    
    #Call leg set function for all legs
    all_legs = []
    for row in rows[1:]:
        rowset = row.find_all('td')
        pleg = [leggify(rowset)]
        all_legs.append(pleg)

    #Remove match separator rows
    for leg in all_legs:
        if (leg[0][1] == 0):
            all_legs.remove(leg)
    
    #Team roster and team name definitions
    teams = psoup.find_all(class_="playerNameHeader")
    team1 = [teams[0].text]
    team2 = [teams[1].text]
    tm1nm = teams[0].text
    tm2nm = teams[1].text

    #Refer to table data on summary page
    homaw = psoup.find_all('td', text=True)

    #Get last 40 and define excluded strings
    homaw = homaw[-40::]
    forbid = ['Premium', 'Recap Sent','Missing Email','Member','&nbsp']

    #Team 2 roster construct
    for pfind in homaw[::-1]:
        row = pfind.text
        if row in forbid:
            continue
        elif row == "Away/Guest Players":
            break
        else:
            team2.append(row)
            continue

    #Team 1 roster construct        
    for pfind in homaw[::-1]:
        row = pfind.text
        if (row in team2) or (row in forbid) or (row == 'Away/Guest Players'):
            continue
        elif row == "Home/Host Players":
            break
        else:
            team1.append(row)

    #Redundancy check
    for player in team1:
        if player in team2:
            print(player, "ON OTHER TEAM")

    #Add aliases to roster

    #Find rows populated with player names in summary table
    players = psoup.find_all('tr')
    versus = []
    for row in players:
        opps = row.find_all(class_='playerName')
        if (opps != []):
            versus.append(opps)
        else:
            continue

    #Differentiate between solo and partner matches, then divide players into teams
    t1 = []
    t2 = []
    for match in versus:
        if (len(match) == 2):
            t1.append(match[0].text)
            t2.append(match[1].text)
        elif (len(match) == 4):
            t1.append(match[0].text)
            t1.append(match[1].text)
            t2.append(match[2].text)
            t2.append(match[3].text)
        else:
            print("I don't know who's playing this match.")

    #Generates tm1 and tm2 spaceless lists
    tm1 = []
    tm2 = []

    #Spaceless from rosters
    for p in team1:
        p = "".join(p.split())
        tm1.append(p)

    for p in team2:
        p = "".join(p.split())
        tm2.append(p)

    #Spaceless aliases from summary table
    for t in t1:
        t = "".join(t.split())
        if t in tm1:
            continue
        else:
            tm1.append(t)

    for t in t2:
        t = "".join(t.split())
        if t in tm2:
            continue
        else:
            tm2.append(t)

    #Generate spaceless player list from PName function
    all_players = PName(all_legs)
    plyrlst = []

    for playr in all_players:
        playr = "".join(playr.split())
        plyrlst.append(playr)

    #Generates the 'Team' column for the dataframe
    teamcol = []
    for player in plyrlst:
        if player in tm1:
            teamcol.append(tm1nm)
        elif player in tm2:
            teamcol.append(tm2nm)
        else:
            teamcol.append('Mysterious Ronin')
            print(player, "not found on any team.")
            
    #Construct game table for match
    gametable = pd.DataFrame(
            {
                "Date": date,
                "Team": teamcol,
                "Player": PName(all_legs),
                "Leg Code": LegCode(all_legs),
                "Game": Game(all_legs),
                "3DA": pd.Series(P3DA(all_legs), dtype="float64"),
                "W/A": LegWin(all_legs)
            })
    
    return gametable

In [14]:
#Input season match data
seasonmatch = pd.read_csv('WCD-19W-Matches.csv')

In [15]:
seasonmatch = seasonmatch['0']

In [16]:
seasonmatch

0      5c3d3935b6e2a72828e2e59b
1      5c4677f2f26c301150bb86cc
2      5c4fb74b1e0efe61e2677a44
3      5c58ce418e82d95b2d56f748
4      5c6b5b037bf62409720964df
                 ...           
247                       44830
248                       46608
249    5c3e9384b6e2a72828e32471
250    5c75f1027bf62409720b6f9a
251    5c88579c6fb2ec48debe94b7
Name: 0, Length: 252, dtype: object

In [17]:
#Get rid of matches not held on official league pages
Season_Matches = []
for match in seasonmatch:
    if (len(match) > 9):
        Season_Matches.append(match)

In [18]:
len(Season_Matches)

248

In [19]:
#Assemble all season legs
Season_Legs = []
for match in Season_Matches:
    Season_Legs.append(MatchTable(match))

I don't know who's playing this match.


In [20]:
#Concatenate into a dataframe
table = pd.concat(Season_Legs)

In [21]:
table.reset_index()

index      Date            Team           Player Leg Code      Game  \
0          0  01-14-19             Win    Matt Cremeens      1.1  501 SIDO   
1          1  01-14-19     Dark Nights      Andrew Weir      1.1  501 SIDO   
2          2  01-14-19             Win    Matt Cremeens      1.2  501 SIDO   
3          3  01-14-19     Dark Nights      Andrew Weir      1.2  501 SIDO   
4          4  01-14-19             Win     Michael Fitt      2.1   Cricket   
...      ...       ...             ...              ...      ...       ...   
16394     63  03-12-19    Cork Busters  Eddie  Palacios      9.1  501 SIDO   
16395     64  03-12-19  Beer B4 Bulls!      Brian McKay      9.2  501 SIDO   
16396     65  03-12-19  Beer B4 Bulls!       Anne Quinn      9.2  501 SIDO   
16397     66  03-12-19    Cork Busters   Nick Georgeson      9.2  501 SIDO   
16398     67  03-12-19    Cork Busters  Eddie  Palacios      9.2  501 SIDO   

        3DA W/A  
0      44.8   0  
1      48.5   W  
2      41.4   0  
3      44.2   W  
4       1.0   0  
...     ...  ..  
16394  62.5   A  
16395  47.3   0  
16396  73.3   0  
16397  75.3   W  
16398  75.0   A  

[16399 rows x 8 columns]

In [22]:
#Output to CSV. Format LEAGUE-SEASON-CONTENTS
table.to_csv("WCD-19W-Legs.csv")